# OpenSearch Neural Search Tutorial
This notebook demonstrates how to set up **Neural Search** (Semantic Search) in OpenSearch.
We will use the built-in **ML Commons** plugin to host a lightweight embedding model (`all-MiniLM-L6-v2`) directly within OpenSearch.

**Prerequisites:**
*   Running OpenSearch Cluster (v2.9+)
*   ML Commons Plugin enabled (default in official images)

**Steps:**
1.  Configure Cluster Settings for ML.
2.  Register a Model Group.
3.  Register and Deploy the Embedding Model.
4.  Create an Ingest Pipeline for Text Embedding.
5.  Create a k-NN Index.
6.  Ingest Data (automatically vectorized).
7.  Perform Neural Search.

In [8]:
import requests
import json
import time

# Configuration
# Adjust these if your OpenSearch is running elsewhere
HOST = 'localhost'
PORT = 19200
PROTOCOL = 'http' # or https
AUTH = ('admin', 'OpenSearch@2024') # Default credentials
VERIFY_SSL = False

BASE_URL = f"{PROTOCOL}::{HOST}:{PORT}"

def run_request(method, endpoint, body=None):
    url = f"{PROTOCOL}://{HOST}:{PORT}/{endpoint}"
    headers = {"Content-Type": "application/json"}
    try:
        if method == "GET":
            resp = requests.get(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "POST":
            resp = requests.post(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "PUT":
            resp = requests.put(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "DELETE":
            resp = requests.delete(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        
        return resp
    except Exception as e:
        print(f"Connection Error: {e}")
        return None

# Check Connection
print(f"Connecting to {BASE_URL}...")
resp = run_request("GET", "")
if resp and resp.status_code == 200:
    print("✅ Connected to OpenSearch")
    print(json.dumps(resp.json(), indent=2))
else:
    print("❌ Failed to connect")
    if resp: print(resp.text)

Connecting to http::localhost:19200...
✅ Connected to OpenSearch
{
  "name": "744516ba3dca",
  "cluster_name": "docker-cluster",
  "cluster_uuid": "VC7Klh05TXOM62fJjRlVSg",
  "version": {
    "distribution": "opensearch",
    "number": "3.3.2",
    "build_type": "tar",
    "build_hash": "6564992150e26aaa62d4522a220dfff5188aeb88",
    "build_date": "2025-10-29T22:24:07.450919802Z",
    "build_snapshot": false,
    "lucene_version": "10.3.1",
    "minimum_wire_compatibility_version": "2.19.0",
    "minimum_index_compatibility_version": "2.0.0"
  },
  "tagline": "The OpenSearch Project: https://opensearch.org/"
}


In [9]:
# 1. Configure Cluster Settings for ML
# These settings allow the model to run on the single node and access external URLs (to download the model).

ml_settings = {
    "persistent": {
        "plugins.ml_commons.only_run_on_ml_node": False,
        "plugins.ml_commons.model_access_control_enabled": True,
        "plugins.ml_commons.native_memory_threshold": 100,
        "plugins.ml_commons.trusted_url_regex": "^https?://.*$" # Allow downloading from HuggingFace
    }
}

print("Configuring ML Settings...")
resp = run_request("PUT", "_cluster/settings", ml_settings)
print(resp.status_code)
print(resp.text)

Configuring ML Settings...
200
{"acknowledged":true,"persistent":{"plugins":{"ml_commons":{"only_run_on_ml_node":"false","model_access_control_enabled":"true","native_memory_threshold":"100","trusted_url_regex":"^https?://.*$"}}},"transient":{}}


In [10]:
# 2. Register Model Group
mg_body = {
    "name": "nlp_model_group",
    "description": "A model group for NLP models"
}
resp = run_request("POST", "_plugins/_ml/model_groups/_register", mg_body)
model_group_id = resp.json().get("model_group_id")
print(f"Model Group ID: {model_group_id}")

# 3. Register Model (all-MiniLM-L6-v2)
# This triggers a download task.
model_body = {
    "name": "huggingface/sentence-transformers/all-MiniLM-L6-v2",
    "version": "1.0.1",
    "model_format": "TORCH_SCRIPT",
    "model_group_id": model_group_id
}

print("Registering Model...")
resp = run_request("POST", "_plugins/_ml/models/_register", model_body)
task_id = resp.json().get("task_id")
print(f"Registration Task ID: {task_id}")

# Poll for Model ID
model_id = None
for i in range(20):
    time.sleep(3)
    status_resp = run_request("GET", f"_plugins/_ml/tasks/{task_id}")
    state = status_resp.json().get("state")
    print(f"Task State: {state}")
    if state == "COMPLETED":
        model_id = status_resp.json().get("model_id")
        print(f"Model Registered! ID: {model_id}")
        break
    elif state == "FAILED":
        print("Registration Failed")
        print(status_resp.text)
        break

# 4. Deploy Model
if model_id:
    print(f"Deploying Model {model_id}...")
    deploy_resp = run_request("POST", f"_plugins/_ml/models/{model_id}/_deploy")
    task_id = deploy_resp.json().get("task_id")
    print(f"Deployment Task ID: {task_id}")
    
    # Poll for Deployment
    for i in range(20):
        time.sleep(3)
        status_resp = run_request("GET", f"_plugins/_ml/tasks/{task_id}")
        state = status_resp.json().get("state")
        print(f"Deployment State: {state}")
        if state == "COMPLETED":
            print("Model Deployed Successfully!")
            break
        elif state == "FAILED":
            print("Deployment Failed")
            print(status_resp.text)
            break

Model Group ID: None
Registering Model...
Registration Task ID: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None
Task State: None


In [11]:
# 5. Create Ingest Pipeline
# This pipeline intercepts documents and converts the 'text' field into embeddings using the model.

pipeline_id = "nlp-ingest-pipeline"
pipeline_body = {
  "description": "An NLP ingest pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          "text": "text_embedding"
        }
      }
    }
  ]
}

if model_id:
    print(f"Creating Pipeline {pipeline_id}...")
    resp = run_request("PUT", f"_ingest/pipeline/{pipeline_id}", pipeline_body)
    print(resp.status_code)
    print(resp.text)
else:
    print("Skipping pipeline creation (No Model ID)")

Skipping pipeline creation (No Model ID)


In [12]:
# 6. Create k-NN Index
index_name = "my-nlp-index"

# Delete if exists
run_request("DELETE", index_name)

index_body = {
  "settings": {
    "index.knn": True,
    "default_pipeline": pipeline_id
  },
  "mappings": {
    "properties": {
      "id": { "type": "text" },
      "text_embedding": {
        "type": "knn_vector",
        "dimension": 384, # Dimension for all-MiniLM-L6-v2
        "method": {
          "name": "hnsw",
          "engine": "lucene",
          "parameters": {
            "m": 16,
            "ef_construction": 128
          }
        }
      },
      "text": { "type": "text" }
    }
  }
}

print(f"Creating Index {index_name}...")
resp = run_request("PUT", index_name, index_body)
print(resp.text)

# 7. Ingest Data
docs = [
    { "text": "A west coast pale ale full of citrus notes", "id": "1" },
    { "text": "A dark stout with coffee and chocolate flavors", "id": "2" },
    { "text": "A refreshing lager with a crisp finish", "id": "3" },
    { "text": "A fruity IPA with tropical vibes", "id": "4" }
]

print("Ingesting Documents...")
for doc in docs:
    run_request("POST", f"{index_name}/_doc", doc)
    
# Refresh to make searchable
run_request("POST", f"{index_name}/_refresh")
print("Data Ingested.")

Creating Index my-nlp-index...
{"acknowledged":true,"shards_acknowledged":true,"index":"my-nlp-index"}
Ingesting Documents...
Data Ingested.
Data Ingested.


In [13]:
# 8. Perform Neural Search
# We search for "something dark to drink" which should match the Stout (id 2) semantically,
# even though the words don't overlap perfectly.

search_query = {
  "query": {
    "neural": {
      "text_embedding": {
        "query_text": "something dark to drink",
        "model_id": model_id,
        "k": 5
      }
    }
  },
  "_source": ["text", "id"]
}

print("--- Neural Search Results ---")
resp = run_request("GET", f"{index_name}/_search", search_query)
hits = resp.json().get("hits", {}).get("hits", [])

for hit in hits:
    score = hit.get("_score")
    source = hit.get("_source")
    print(f"Score: {score:.4f} | Text: {source.get('text')}")

--- Neural Search Results ---
